In [1]:
import parallel_gpt
import asyncio
import pandas as pd 
import random

# chat api를 병렬처리하는 경우

In [2]:
# 프롬프트가 리스트 형태로 미리 준비되어 있어야 함
prompt_list = [f'calculate {i} * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.' for i in range(30)]

# 프롬프트 생성 확인
random.sample(prompt_list, 5)

['calculate 28 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.',
 'calculate 1 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.',
 'calculate 4 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.',
 'calculate 29 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.',
 'calculate 16 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.']

In [3]:
# 파라미터 설정
MODEL = "gpt-4o"
TEMPERATURE = .0
API_KEY = "YOUR_API_KEY"
JSON_MODE = True # JSON 모드 사용 시에는 프롬프트에 JSON 이라는 단어가 들어가 있어야 함
TIMEOUT = 180 # 최대 응답 대기 (sec)
SEED = 42
MAX_ATTEMPTS = 30 # 에러가 발생할 경우 재시도 하는 횟수

# RPM TPM 은 계정의 Tier에 따라 설정 현재 예시에서는 RPM 10000, TPM 2000000 사용
LATENCY = 0.95
RPM = 10000 * LATENCY 
TPM = 2000000 * LATENCY

In [4]:
# 병렬처리 호출
chat_result_list = asyncio.run(parallel_gpt.process_api_requests_from_prompt_list(
    prompt_list=prompt_list,
    model=MODEL,
    temperature=TEMPERATURE,
    api_key=API_KEY,
    request_url="https://api.openai.com/v1/chat/completions",
    json_mode=JSON_MODE,
    desc="chat usage sample...!",
    timeout=TIMEOUT,
    seed=SEED,
    max_attempts=MAX_ATTEMPTS,
    max_requests_per_minute=RPM,
    max_tokens_per_minute=TPM,
))

INFO:root:chat usage sample...! Starting completions >>> #0
INFO:root:chat usage sample...! Starting completions >>> #1
INFO:root:chat usage sample...! Starting completions >>> #2
INFO:root:chat usage sample...! Starting completions >>> #3
INFO:root:chat usage sample...! Starting completions >>> #4
INFO:root:chat usage sample...! Starting completions >>> #5
INFO:root:chat usage sample...! Starting completions >>> #6
INFO:root:chat usage sample...! Starting completions >>> #7
INFO:root:chat usage sample...! Starting completions >>> #8
INFO:root:chat usage sample...! Starting completions >>> #9
INFO:root:chat usage sample...! Starting completions >>> #10
INFO:root:chat usage sample...! Starting completions >>> #11
INFO:root:chat usage sample...! Starting completions >>> #12
INFO:root:chat usage sample...! Starting completions >>> #13
INFO:root:chat usage sample...! Starting completions >>> #14
INFO:root:chat usage sample...! Starting completions >>> #15
INFO:root:chat usage sample...! St

In [5]:
# 병렬처리하면 아래와 같이 첫번 째 요소에 순서와 두번째에 gpt api response를 얻을 수 있음
chat_result_list[0]


(0,
 [{'model': 'gpt-4o',
   'messages': [{'role': 'user',
     'content': 'calculate 0 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.'}],
   'temperature': 0.0,
   'max_tokens': 3000,
   'frequency_penalty': 0,
   'seed': 42,
   'response_format': {'type': 'json_object'}},
  {'id': 'chatcmpl-9begmFoy4DGigCb80DQMdGNogf9N2',
   'object': 'chat.completion',
   'created': 1718761572,
   'model': 'gpt-4o-2024-05-13',
   'choices': [{'index': 0,
     'message': {'role': 'assistant', 'content': '{\n  "result": 0\n}'},
     'logprobs': None,
     'finish_reason': 'stop'}],
   'usage': {'prompt_tokens': 35, 'completion_tokens': 9, 'total_tokens': 44},
   'system_fingerprint': 'fp_f4e629d0a5'}])

In [6]:
# 각 결과를 아래와 같이 파싱하고 데이터 프레임으로 변환하여 활용
result_list = [None] * len(prompt_list)
raw_result_list = [None] * len(prompt_list)
input_token_list = [None] * len(prompt_list)
output_token_list = [None] * len(prompt_list)
total_token_list = [None] * len(prompt_list)
price_list = [None] * len(prompt_list)

for chat_result in chat_result_list:
    if chat_result is not None:
        raw_result_list[chat_result[0]] = chat_result[1]
        result_list[chat_result[0]] = parallel_gpt.parse_json_string(chat_result[1][1]['choices'][0]['message']['content'], ['result'])['result']
        input_token_list[chat_result[0]] = chat_result[1][1]['usage']['prompt_tokens']
        output_token_list[chat_result[0]] = chat_result[1][1]['usage']['completion_tokens']
        total_token_list[chat_result[0]] = chat_result[1][1]['usage']['prompt_tokens'] + chat_result[1][1]['usage']['completion_tokens']
        price_list[chat_result[0]] = (chat_result[1][1]['usage']['prompt_tokens'] * 0.005 / 1000) + (chat_result[1][1]['usage']['completion_tokens'] * 0.015 / 1000)

pd.DataFrame({
    "prompt": prompt_list,
    "result": result_list,
    "raw_result": raw_result_list,
    "input_token": input_token_list,
    "output_token": output_token_list,
    "total_token": total_token_list,
    "price": price_list,
})

,prompt,result,raw_result,input_token,output_token,total_token,price
0,calculate 0 * 3\nRespond in JSON format and in...,0,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
1,calculate 1 * 3\nRespond in JSON format and in...,3,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
2,calculate 2 * 3\nRespond in JSON format and in...,6,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
3,calculate 3 * 3\nRespond in JSON format and in...,9,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
4,calculate 4 * 3\nRespond in JSON format and in...,12,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
5,calculate 5 * 3\nRespond in JSON format and in...,15,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
6,calculate 6 * 3\nRespond in JSON format and in...,18,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
7,calculate 7 * 3\nRespond in JSON format and in...,21,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
8,calculate 8 * 3\nRespond in JSON format and in...,24,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031
9,calculate 9 * 3\nRespond in JSON format and in...,27,"[{'model': 'gpt-4o', 'messages': [{'role': 'us...",35,9,44,0.00031


# embedding을 병렬처리하는 경우

In [7]:
import parallel_gpt
import asyncio
import pandas as pd 
import random

In [8]:
# 임베딩 할 텍스트 리스트가 준비되어 있어야 함
embedding_text_list = [f'calculate {i} * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.' for i in range(30)]

# 임베딩 텍스트 리스트 확인
random.sample(embedding_text_list, 5)

['calculate 27 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.',
 'calculate 14 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.',
 'calculate 25 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.',
 'calculate 16 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.',
 'calculate 0 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.']

In [9]:
# 파라미터 설정
MODEL = "text-embedding-3-large"
API_KEY = "YOUR_API_KEY"
TIMEOUT = 180 # 최대 응답 대기 (sec)
MAX_ATTEMPTS = 30 # 에러가 발생할 경우 재시도 하는 횟수

# RPM TPM 은 계정의 Tier에 따라 설정 현재 예시에서는 RPM 10000, TPM 2000000 사용
LATENCY = 0.95
RPM = 10000 * LATENCY 
TPM = 10000000 * LATENCY

embedding_result_list = asyncio.run(parallel_gpt.process_api_requests_from_prompt_list(
        prompt_list=embedding_text_list,
        model=MODEL,
        api_key=API_KEY,
        request_url = "https://api.openai.com/v1/embeddings",
        max_attempts=MAX_ATTEMPTS,
        max_requests_per_minute=RPM,
        max_tokens_per_minute=TPM,
        desc="embedding",
        timeout=TIMEOUT
))

INFO:root:embedding Starting embeddings >>> #0
INFO:root:embedding Starting embeddings >>> #1
INFO:root:embedding Starting embeddings >>> #2
INFO:root:embedding Starting embeddings >>> #3
INFO:root:embedding Starting embeddings >>> #4
INFO:root:embedding Starting embeddings >>> #5
INFO:root:embedding Starting embeddings >>> #6
INFO:root:embedding Starting embeddings >>> #7
INFO:root:embedding Starting embeddings >>> #8
INFO:root:embedding Starting embeddings >>> #9
INFO:root:embedding Starting embeddings >>> #10
INFO:root:embedding Starting embeddings >>> #11
INFO:root:embedding Starting embeddings >>> #12
INFO:root:embedding Starting embeddings >>> #13
INFO:root:embedding Starting embeddings >>> #14
INFO:root:embedding Starting embeddings >>> #15
INFO:root:embedding Starting embeddings >>> #16
INFO:root:embedding Starting embeddings >>> #17
INFO:root:embedding Starting embeddings >>> #18
INFO:root:embedding Starting embeddings >>> #19
INFO:root:embedding Starting embeddings >>> #20
IN

In [10]:
# 병렬처리하면 아래와 같이 첫번 째 요소에 순서와 두번째에 embedding api response를 얻을 수 있음
embedding_result_list[0]

(0,
 [{'model': 'text-embedding-3-large',
   'input': 'calculate 0 * 3\nRespond in JSON format and include the result with the key "result" and the value as the result.'},
  {'object': 'list',
   'data': [{'object': 'embedding',
     'index': 0,
     'embedding': [0.0003121347,
      -0.046130132,
      -0.0020390754,
      0.033972893,
      0.019878354,
      -0.01999066,
      0.028582152,
      0.080692634,
      -0.0010704269,
      0.010290137,
      0.021984112,
      0.031698674,
      -0.012318683,
      0.004773051,
      0.008942451,
      0.0031691655,
      -0.045680903,
      -0.0019794123,
      -0.02832946,
      -0.011609744,
      -0.012697719,
      0.002728711,
      -0.00089714053,
      0.018446438,
      -0.004860791,
      0.011378111,
      0.013245217,
      0.014698189,
      0.0154281845,
      -0.016803946,
      -0.018123554,
      0.0076719774,
      0.0038605558,
      -0.04225554,
      -0.014284057,
      -0.020664502,
      0.02772581,
      0.0119536

In [11]:
# 각 결과를 아래와 같이 파싱하고 데이터 프레임으로 변환하여 활용
input_content_list = [None] * len(embedding_text_list)
result_list = [None] * len(embedding_text_list)
input_token_list = [None] * len(embedding_text_list)
price_list = [None] * len(embedding_text_list)

for embedding_result in embedding_result_list:
    if embedding_result is not None:
        input_content_list[embedding_result[0]] = embedding_result[1][0]['input']
        result_list[embedding_result[0]] = embedding_result[1][1]['data'][0]['embedding']
        input_token_list[embedding_result[0]] = embedding_result[1][1]['usage']['total_tokens']
        price_list[embedding_result[0]] = embedding_result[1][1]['usage']['total_tokens'] * 0.00000013

pd.DataFrame({
    "input_content": input_content_list,
    "result": result_list,
    "input_token": input_token_list,
    "price": price_list,
})

,input_content,result,input_token,price
0,calculate 0 * 3\nRespond in JSON format and in...,"[0.0003121347, -0.046130132, -0.0020390754, 0....",28,0.000004
1,calculate 1 * 3\nRespond in JSON format and in...,"[0.009739864, -0.032784324, -0.0017534212, 0.0...",28,0.000004
2,calculate 2 * 3\nRespond in JSON format and in...,"[0.0022471845, -0.03435882, -0.0039203223, 0.0...",28,0.000004
3,calculate 3 * 3\nRespond in JSON format and in...,"[0.00022599146, -0.030189373, -0.0033842972, 0...",28,0.000004
4,calculate 4 * 3\nRespond in JSON format and in...,"[-0.001717173, -0.024962, -0.0029649278, 0.039...",28,0.000004
5,calculate 5 * 3\nRespond in JSON format and in...,"[-0.005395843, -0.022861974, -0.004930265, 0.0...",28,0.000004
6,calculate 6 * 3\nRespond in JSON format and in...,"[0.0012079233, -0.031725597, -0.0039004064, 0....",28,0.000004
7,calculate 7 * 3\nRespond in JSON format and in...,"[0.0014187947, -0.032032017, -0.0008867467, 0....",28,0.000004
8,calculate 8 * 3\nRespond in JSON format and in...,"[-0.0042981203, -0.028737191, -0.000862133, 0....",28,0.000004
9,calculate 9 * 3\nRespond in JSON format and in...,"[0.0085241245, -0.0151292775, -0.0014931123, 0...",28,0.000004
